## Some simple examples with PRA-socp
#### Projection and rescaling algorithm for second-order conic systems

In [1]:
import numpy as np
import time
import pandas as pd
from PRAalgos import Cone,PRA,PRAcpversion,NaiveInstance,ControlledInstance
from PRAtests import experiments,comparePRA,comparison

#### Create a cone, a naive random instance, and center of the cone

In [2]:
r = 10; d = 10 ; dim = np.ones(r)*d ; dim = dim.astype(int);
K = Cone(dim) ; n = sum(K.dim) ; 
m = n/2 ; m = m.astype(int) ;
A,AA = NaiveInstance(m,sum(K.dim)) ; 
u0 = np.zeros(n) ; u0[K.sdim] = 1 ; u0 = u0/(2*sum(u0))

#### Run PRA algorithm and report results

In [3]:
feas,xL,xLperp,k,Total = PRA(A, AA, K, u0)
print('number of rescalings steps = ', k, ',  number of iterations = ',Total)
if (feas == 1): 
    print('found solution in L \cap K') 
    print('norm(xL) = ',str(np.linalg.norm(xL)))    
    print('norm(A@xL) = ', str(np.linalg.norm(A@xL)))
elif (feas == 2): 
    print('found solution in Lperp \cap K') 
    print('norm(xLperp) = ',str(np.linalg.norm(xLperp)))    
    print('norm(AA@xLperp) = ', str(np.linalg.norm(AA@xLperp)))
else:
    print('PRA failed')

number of rescalings steps =  0 ,  number of iterations =  3
found solution in L \cap K
norm(xL) =  1.0
norm(A@xL) =  7.692422894854056e-16


#### Compare with cvxpy version

In [5]:
xL,xLperp,feas = PRAcpversion(A,AA,K,solver = 'MOSEK')
if (feas == 1): 
    print('found solution in L \cap K') 
    print('norm(xL) = ',str(np.linalg.norm(xL)))
    print('norm(A@xL) = ', str(np.linalg.norm(A@xL)))
elif (feas == 2): 
    print('found solution in Lperp \cap K') 
    print('norm(xLperp) = ',str(np.linalg.norm(xLperp)))
    print('norm(AA@xLperp) = ', str(np.linalg.norm(AA@xLperp)))
else:
    print('PRAcpversion failed')     

found solution in L \cap K
norm(xL) =  0.9999999999999999
norm(A@xL) =  5.948553479709031e-13


#### Run some experiments to compare PRA with GUROBI, ECOS, SCS on controlled instances

In [6]:
deltaset = (1,0.1,0.01,0.001) ; N = 5
rset = (5,10,20) ; n = 100
compsuccess,compCPU,largestnorm,smallestminev = comparison(rset,n,deltaset,N)

Running experiments for N = 5, r = 5, n = 100, delta = 1
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-12


/Users/javipena/opt/anaconda3/lib/python3.9/site-packages/cvxpy/problems/problem.py:1294: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Running experiments for N = 5, r = 10, n = 100, delta = 1
Running experiments for N = 5, r = 20, n = 100, delta = 1
Running experiments for N = 5, r = 5, n = 100, delta = 0.1
Running experiments for N = 5, r = 10, n = 100, delta = 0.1
Running experiments for N = 5, r = 20, n = 100, delta = 0.1
Running experiments for N = 5, r = 5, n = 100, delta = 0.01
Running experiments for N = 5, r = 10, n = 100, delta = 0.01
Running experiments for N = 5, r = 20, n = 100, delta = 0.01
Running experiments for N = 5, r = 5, n = 100, delta = 0.001
Running experiments for N = 5, r = 10, n = 100, delta = 0.001
Running experiments for N = 5, r = 20, n = 100, delta = 0.001


 Number of instances solved successfully

        (1,5,100)  (1,10,100)  (1,20,100)  (0.1,5,100)  (0.1,10,100)  \
PRA             5           5           5            5             5   
GUROBI          5           5           5            5             5   
MOSEK           5           5           5            5             5   
ECOS  

#### Run experiments as in the paper

In [ ]:
N = 5
lset = (3, 5, 9, 10, 12)
dset = lset
deltaset = (1.000, 0.500, 0.20, 0.1)
limdim = 200
dsum,dfResult = experiments(lset,dset,deltaset,N,limdim)